In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [2]:
# 解析高德地图API网址信息
def get_latlng_gaode(address,ak):
    ak= ak
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [3]:
def get_receiver_lnglat_gaode(address):
    start_time = time.time()
    re_List_gaode = []
    for b in address:
        print(b)
        re_Dict_gaode = {}
        try:
            temp=get_latlng_gaode(b,ak_gaode)
        except requests.exceptions.ConnectionError: # HTTPError
            re_Dict_gaode["re_location_gaode"] = "请求出错"         
        else:
            if (temp['count'] == "1"):
                if('location' in temp['geocodes'][0]):
                    re_Dict_gaode["re_location_gaode"] = temp['geocodes'][0]['location']
                else:
                    pass
                if ('formatted_address' in temp['geocodes'][0]):
                    re_Dict_gaode["re_formatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                else:
                    pass

            elif (temp['count'] == "0"):
                pass
            elif (temp['status'] == "0"):
                pass
            else:
                pass
        re_List_gaode.append(re_Dict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("高德总用时：",end_time-start_time)
    return re_List_gaode

In [4]:
def parse_gaode(df,address):
    # 解析高德
    list_gaode = get_receiver_lnglat_gaode(address)
    # df化(高德)
    df_gaode = pd.DataFrame(list_gaode)
    data_gaode = pd.concat([df,df_gaode],axis=1)
    print("data_gaode:" + str(data_gaode.shape))
    return data_gaode

In [5]:
# ak码
# ak_gaode = 'c71d9eda293d20db64955275557d92d4'
ak_gaode = "1c2682317206ccaa18ec2487bb45d880"
ak_baidu = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'

In [6]:
# 导入数据
date = "1902"
fileName = "testJDXX.xlsx"
pinyin = "JDXX"
mainpath = "D:\\work\\basic\\carriers\\"
inputpath = mainpath + date + "\\"
data = pd.read_excel(inputpath + fileName)

In [7]:
# 导出数据路径
outpath_gaode = mainpath + date + "\\output\\gaode\\"
name_ANSI_gaode = date + "Info_lnglat_" + pinyin + "_gaode.xlsx"
name_UTF8_gaode = date + "Info_lnglat_" + pinyin + "_gaode.csv"

outpath_baidu = mainpath + date + "\\output\\baidu\\"
name_ANSI_baidu = date + "Info_lnglat_" + pinyin + "_baidu.xlsx"
name_UTF8_baidu = date + "Info_lnglat_" + pinyin + "_baidu.csv"


In [8]:
data["所在省"].value_counts()

广东    3
辽宁    1
湖北    1
重庆    1
Name: 所在省, dtype: int64

### 选取有效字段

In [9]:
df = data[["所在省","所在市","收货地址","订单编号","承运商","所在县"]]

In [10]:
df["address_all"]= df["所在省"] +"省"+ df["所在市"] + df["收货地址"]

d:\python\py35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df1 = df[df["所在省"].isin(["广东"])].reset_index(drop=True)
# df2 = df[df["所在省"].isin(["山东"])].reset_index(drop=True)
# df3 = df[df["所在省"].isin(["河南"])].reset_index(drop=True)

# df4 = df[df["所在省"].isin(["江苏"])].reset_index(drop=True)
# df5 = df[df["所在省"].isin(["河北"])].reset_index(drop=True)
# df6 = df[df["所在省"].isin(["湖北"])].reset_index(drop=True)
# df7 = df[df["所在省"].isin(["北京"])].reset_index(drop=True)
# df8 = df[df["所在省"].isin(["湖南"])].reset_index(drop=True)

# df9 = df[df["所在省"].isin(["浙江","上海"])].reset_index(drop=True)
# df10 = df[df["所在省"].isin(["四川","云南","广西"])].reset_index(drop=True)
# df11 = df[df["所在省"].isin(["辽宁","黑龙江","安徽"])].reset_index(drop=True)
# df12 = df[df["所在省"].isin(["陕西","山西"])].reset_index(drop=True)

# df13 = df[df["所在省"].isin(["吉林","内蒙古","重庆","江西","福建","贵州","甘肃","天津","宁夏","海南","青海","西藏","新疆"])].reset_index(drop=True)

### 文件分组并检查

In [12]:
print("data:" + str(data.shape))
print("df:" + str(df.shape))

data:(6, 49)
df:(6, 7)


In [13]:
data1_gaode = parse_gaode(df1,df1["address_all"])

广东省深圳市中山园路建工村保障性住房12栋1105
广东省潮州市高堂镇邦湖村
广东省广州市竹料镇沙亭岗村中国O3产业园
高德总用时： 0.17255449295043945
data_gaode:(3, 9)


In [14]:
data1_gaode

,所在省,所在市,收货地址,订单编号,承运商,所在县,address_all,re_formatted_address_gaode,re_location_gaode
0,广东,深圳市,中山园路建工村保障性住房12栋1105,600101844000008319022502045182,京东线下,南山区,广东省深圳市中山园路建工村保障性住房12栋1105,广东省深圳市南山区建工村保障性住房|12栋,"113.927380,22.564155"
1,广东,潮州市,高堂镇邦湖村,690000000001008319021822065408,京东线下,饶平县,广东省潮州市高堂镇邦湖村,广东省潮州市饶平县高堂镇,"116.940537,23.706640"
2,广东,广州市,竹料镇沙亭岗村中国O3产业园,600101844000008319022201829436,京东线下,白云区,广东省广州市竹料镇沙亭岗村中国O3产业园,NaN,NaN


In [ ]:
data_gaode = pd.concat([data1_gaode,data2_gaode,data3_gaode,data4_gaode,data5_gaode,data6_gaode,data7_gaode,
                        data8_gaode,data9_gaode,data10_gaode,data11_gaode,data12_gaode,data13_gaode])

In [ ]:
# 合表
split_gaode = pd.DataFrame((x.split(',') for x in data_gaode['re_location_gaode']), index=data_gaode.index, columns=['re_siteLng_gaode','re_siteLat_gaode'])
data_gaode = pd.merge(data_gaode,split_gaode,left_index=True,right_index=True)


In [ ]:
# data_baidu = parse_baidu(df,df["address_all"])

In [ ]:
# 检查合并后结构
print("data:" + str(data.shape))
print("df:" + str(df.shape))
print("data_gaode:" + str(data_gaode.shape))
# print("data_baidu:" + str(data_baidu.shape))

In [ ]:
# data_gaode.to_csv(outpath_gaode + name_UTF8_gaode,index=False)
# data_baidu.to_csv(outpath_baidu + name_UTF8_baidu,index=False)

In [ ]:
data_gaode.head()